# Модуль 4: Рекуррентные нейронные сети (RNN)

## Теория

### Что такое рекуррентные нейронные сети (RNN)?

**Рекуррентные нейронные сети (RNN)** — это тип нейронных сетей, предназначенных для работы с последовательными данными, такими как временные ряды, текст, аудио, видео и др. В отличие от стандартных полносвязных и сверточных сетей, RNN имеет возможность "помнить" информацию из предыдущих состояний, что позволяет учитывать контекст.  
  
Основная особенность RNN заключается в том, что они используют рекуррентную связь. На каждом шаге RNN получает входные данные и состояние из предыдущего шага, что позволяет модели обрабатывать последовательности данных.

### Основные типы RNN

**Простая RNN**: Первая версия сети, которая использует рекуррентные связи, но подвержена проблемам с затухающими и взрывными градиентами, что делает обучение долгим и сложным в случае длинных последовательностей.

**LSTM (Long Short-Term Memory)**: Это более сложный тип RNN, который введен для решения проблем, связанных с простыми RNN. LSTM имеет специальные структуры — ячейки (cells), которые помогают сохранять информацию на длительное время благодаря механизмам управления, таким как "входной", "выходной" и "выходной" гейты. Это делает LSTM более эффективными для работы с длинными последовательностями.

**GRU (Gated Recurrent Unit)**: Это более упрощенная версия LSTM, которая также использует механизмы врат, но с меньшим количеством параметров, что позволяет быстро обучаться и использовать меньше ресурсов.

### Применение RNN

Основное применение RNN: Обработка последовательных данных. Например:
1. Обработка естественного языка (NLP): машинный перевод, генерация текста, анализSentiment.
2. Временные ряды: предсказание финансовых рынков, анализ сигнала, моделирование погодных условий.
3. Аудио и музыка: генерация музыки, автоматическое распознавание речи.

### Определения

**Затухающие градиенты** — это проблема, возникающая в глубоком обучении, когда градиенты, вычисляемые для обновления весов, становятся очень маленькими (близкими к нулю) по мере обратного распространения ошибки через множество слоев. Это приводит к тому, что:
 - Невозможность обучения: При очень маленьких градиентах обновление весов происходит очень медленно, особенно для нижних (или "более ранних") слоев сети. В результате они почти не обучаются, и сеть не может принимать правильные решения.
 - Проблемы с долгосрочной памятью: Это делает RNN, которые должны запоминать информацию на протяжении долгих последовательностей, менее эффективными, так как они теряют способность запоминать важные зависимости.
  
**Взрывные градиенты** — это противоположное явление, когда градиенты становятся очень большими по мере их распространения через сеть. Это может привести к:
 - Нестабильности обучения: Большие градиенты могут привести к резким изменениям весов, что вызывает колебания в значениях функции потерь и могут даже привести к "разрушению" модели.
 - Невозможность сходимости: Сеть может начать "плясать" около точки без сходимости, так как веса становятся слишком большими.

**Последовательные данные** — это данные, где порядок элементов имеет значение. Они представляют собой последовательности, в которых каждый элемент связан с предыдущими и последующими. Например:
1. Временные ряды: Данные, собранные в последовательности во времени. Например, температура за день, цены акций, количество продаж по дням.
2. Текст: Слова или символы, которые имеют последовательный порядок. Например, предложения, абзацы, языковые модели.
3. Аудио: Звуковые волны, представленные во временной последовательности. Например, звук речи или музыки.
4. Видео: Последовательность кадров, где каждый кадр зависит от предыдущего. Например, фильмы, записанные видеопотоки.
5. Биологические последовательности: Данные в геномике, такие как последовательности нуклеотидов в ДНК.
  
**Непоследовательные данные**: Могут включать статические данные, такие как изображения или таблицы, где порядок элементов не имеет значения.
  
**Данные со структурами**: Могут включать графы, деревья и другие сложные структуры, в которых элементы могут иметь различные уровни взаимосвязи.

## Задачи

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

from datasets import load_dataset
from transformers import AutoTokenizer

from tqdm import tqdm

In [2]:
from huggingface_hub import login
import os

login(token=os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
# Указание в качестве девайса GPU если доступна
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### Функции

In [4]:
def train_model(
        model: nn.Module,
        num_epochs: int,
        optimizer: torch.optim,
        criterion: torch.nn,
        train_dataloader: DataLoader
    ):
    """
    Функция для обучения модели.
    
    Args:
        model (nn.Module): модель для обучения
        num_epochs (int): кол-во итераций обучения
        optimizer (torch.optim): оптимизатор
        criterion (torch.nn): функция потерь
        train_dataloader (torch.utils.data.DataLoader): загрузчик данных

    Output:
        loss_list (list): список значений функции потерь за каждый батч
        acc_list (list): список точностей за каждый батч

    """
    loss_list = []
    acc_list = []

    # Кол-во обучающих премеров
    train_amount = len(train_dataloader)

    # Загрузка модели на GPU
    model.train().to(device)

    # Обучение
    for epoch in range(num_epochs):
        total_loss = 0
        total_acc = 0
        
        for batch in tqdm(train_dataloader):
            # Обнуление градиентов
            optimizer.zero_grad()

            # Перевод всех данных на GPU
            data = batch["input_ids"].to(device)
            targets = batch["labels"].to(device)

            # Предсказания модели
            outputs = model(data)

            # Расчет функции потерь
            loss = criterion(outputs, targets)

            # Прибавление функции потерь к общей потере за эпоху
            total_loss += loss.item()

            # Вычисление градиентов
            loss.backward()

            # Отпимизация весов на основе расчитанных градиентов
            optimizer.step()

            # Расчет точности (accuracy)
            total = targets.shape[0]
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == targets).sum().item()
            total_acc += correct / total
        
        # Расчет средней точности за эпоху
        avg_loss = total_loss / train_amount
        loss_list.append(avg_loss)

        # Расчет средней точности за эпоху
        avg_accuracy = total_acc / train_amount
        acc_list.append(avg_accuracy)

        # Вывод информации об эпохе обучения модели
        print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {avg_loss:.2f}, Accuracy: {avg_accuracy:.2f}')
        print("=" * 100)
        print()

    return loss_list, acc_list


In [24]:
def eval_model(
    model: nn.Module,
    test_dataloader: DataLoader  
):
    """
    Функция для оценки обученной модели.

    Args:
        model (nn.Module): обученная модель
        test_dataloader (DataLoader): даталодер для тестовых данных

    Output:
        None
    """

    # Оценка обученной модели
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in tqdm(test_dataloader):
            # Перевод всех данных на GPU
            data = batch["input_ids"].to(device)
            targets = batch["labels"].to(device)
            
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.shape[0]
            correct += (predicted == targets).sum().item()

        print('Test Accuracy of the model on the {} batch test texts: {} %'.format(len(test_dataloader), (correct / total) * 100))

In [6]:
def tokenize_text(example):
    example["input_ids"] = tokenizer(example["text"], truncation=True, max_length=256)["input_ids"]
    return example

### Реализовать RNN для предсказания временных рядов

С помощью синтетических или реальных данных, создать простую RNN для предсказания следующего значения в последовательности.

В качестве примера задача: Классификация отзывов на фильмы c Кинопоиска.

In [7]:
# Загрузка датасета с отзывами
dataset = load_dataset("ai-forever/kinopoisk-sentiment-classification")

In [8]:
# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-conversational')

In [9]:
# Предобработка текстового датасета
# 1. Токенизация предложений с помощью Токенизатора
tokenized_dataset = dataset.map(tokenize_text, batched=True,
                                remove_columns=["label_text"])

In [10]:
# 2. Удаление лишнего класса ('1' - нейтральный отзыв)
tokenized_dataset = tokenized_dataset.filter(lambda example: example["label"] != 1)

In [13]:
# 3. Переименование класса '2' в '1'
def rename_label(example):
    if example["label"] == 2:
        example["label"] = 1
    return example

tokenized_dataset = tokenized_dataset.map(rename_label)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
# 4. Визуализация данных
class_dict = {0: "Отрицательный", 1: "Положительный"}
text = tokenized_dataset["train"][0]["text"]
label = class_dict[tokenized_dataset["train"][0]["label"]]
print("Пример текста: {}\n".format(text))
print("Класс: {}".format(label))

Пример текста: Если честно, меня не очень впечатлила новость, о том, что Гай Ричи собирается снять фильм, о Шерлоке Холмсе. Подумал — да это же будет: Карты, деньги, два ствола — у Холмса и у Ватсона. Но затем по мере появления трейлеров, и большей информации поменял своё отношение.
«Шерлок Холмс» — последняя картина, на которую я планировал пойти в этом году. Жутко боялся, что она меня разочарует т. к. перед этим были расстроившие «Безумный спецназ», «Так себе каникулы» и немного 2012. Но Холмс полностью оправдал доверия.
Сюжет
В ленте динамичный, а главное интересный и захватывающий сюжет, что в последнее время не так уж и часто. Новый Холмс не поход классический образ представленный в картинах Игоря Масленникова, но это не портит его образ. Он больше подобен на Тони Старку из «Железного человека» или Грегори Хаусу из сериала «Доктор Хаус». Как и они, он весьма харизматичен, слегка безумен, но гениален в своём любимом деле.
На первый взгляд сюжет портит сверхъестественное восстание и

In [17]:
# 5. Определение train, test, val datasets
train_ds = tokenized_dataset["train"].select(range(300))  # Первые 300 примеров
val_ds = tokenized_dataset["validation"].select(range(50))  # Следующие 50 примеров для валидации
test_ds = tokenized_dataset["train"].select(range(50))  # Следующие 50 примеров для тестирования

In [18]:
# 6. Оставляем только колонки input_ids и label
train_ds = train_ds.select_columns(["input_ids", "label"])
val_ds = val_ds.select_columns(["input_ids", "label"])
test_ds = test_ds.select_columns(["input_ids", "label"])

In [19]:
from transformers import DataCollatorWithPadding

# 7. Создание DataLoaders
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_ds, batch_size=32, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=32, collate_fn=collate_fn)

In [20]:
# 8. Визуализация размерности данных
for batch in train_loader:
    data = batch["input_ids"]
    targets = batch["labels"]
    print(data.shape, targets.shape)
    break

torch.Size([32, 256]) torch.Size([32])


In [21]:
# 9. Определение модели RNN
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, n_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # Слой RNN
        self.rnn = nn.RNN(input_size=embed_size, hidden_size=hidden_size, num_layers=1, batch_first=True)

        # Полносвязный слой
        self.fc = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        # Прогон через слой эмбеддинга
        x = self.embedding(x)
        
        # Прогон через RNN
        output, hidden = self.rnn(x)
        
        # Прогон через полносвязный слой
        out = self.fc(output[:, -1])
        return out
    

In [22]:
# 10. Обучение модели
model = RNN(vocab_size=tokenizer.vocab_size, embed_size=256, hidden_size=128, n_classes=2)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение
loss_list, acc_list = train_model(model, 5, optimizer, criterion, train_loader)

100%|██████████| 10/10 [00:00<00:00, 21.04it/s]


Epoch: 1/5, Loss: 0.78, Accuracy: 0.51



100%|██████████| 10/10 [00:00<00:00, 42.91it/s]


Epoch: 2/5, Loss: 0.69, Accuracy: 0.58



100%|██████████| 10/10 [00:00<00:00, 42.04it/s]


Epoch: 3/5, Loss: 0.60, Accuracy: 0.75



100%|██████████| 10/10 [00:00<00:00, 42.85it/s]


Epoch: 4/5, Loss: 0.56, Accuracy: 0.69



100%|██████████| 10/10 [00:00<00:00, 42.55it/s]

Epoch: 5/5, Loss: 0.45, Accuracy: 0.79



In [25]:
# 11. Оценка модели
eval_model(model, test_loader)

100%|██████████| 2/2 [00:00<00:00, 58.82it/s]

Test Accuracy of the model on the 2 batch test texts: 76.0 %


In [30]:
# 12. Проверка модели на случайном примере
text = "Фильм отстой. Не советую смотреть."
model.eval()

with torch.no_grad():
    inputs = tokenizer(text, return_tensors="pt")["input_ids"].to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    print(f"Текст: {text}")
    print(f"Предсказание: {class_dict[predicted.item()]}")
    prob = torch.max(torch.softmax(outputs, dim=1)).item()
    print(f"Вероятность ответа: {prob * 100:.2f}%")

Текст: Фильм отстой. Не советую смотреть.
Предсказание: Отрицательный
Вероятность ответа: 78.38%


**Результат** - рассмотрена простейшая нейросеть архитектуры RNN. После обучения на 300 обучающих данных имеет точность в 76%. 